# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [9]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Ask for parameters

In [33]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [34]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $i$: Index for thermal units, $i = 1, 2, ..., 6$
* $t$: Index for time periods, $t = 1, 2, ..., 15$ 

## Parameters:

* $SU_i$: Startup cost of unit $i$ (€) 
* $SD_i$: Shutdown cost of unit $i$ (€)
* $FC_i$: Fixed cost of unit $i$ (€) 
* $VC_{i,t}$: Variable cost of unit $i$ at time $t$ (€/MWh)
* $P_{i}^{min}$: Minimum power output of unit $i$ (MW)
* $P_{i}^{max}$: Maximum power output of unit $i$ (MW)
* $RU_i$: Maximum power increment of unit $i$ from one period to the next (MW) 
* $RD_i$: Maximum power decrement of unit $i$ from one period to the next (MW)
* $D_t$: Total power demand at time $t$ (MW)

## Variables:

* $P_{i,t}$: Power output of unit $i$ at time $t$ (MW)
* $U_{i,t}$: Binary variable indicating if unit $i$ is running at time $t$ (1 if running, 0 otherwise)
* $SU_{i,t}$: Binary variable indicating if unit $i$ is started up at time $t$ (1 if started, 0 otherwise)
* $SD_{i,t}$: Binary variable indicating if unit $i$ is shut down at time $t$ (1 if shut down, 0 otherwise) 


# 2. Ask for objective

In [35]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [36]:
Markdown(response2.text)

$$
\text{Minimize} \sum_{i=1}^{6} \sum_{t=1}^{15} (FC_i \cdot U_{i,t} + VC_{i,t} \cdot P_{i,t} + SU_i \cdot SU_{i,t} + SD_i \cdot SD_{i,t}) 
$$


# 3. Ask for constraints

In [39]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [40]:
Markdown(response3.text)

## Constraints:

**1. Power balance:** The total power generated must meet demand plus a 10% reserve margin.
   
   $$ \sum_{i=1}^{6} P_{i,t} \geq 1.1 \cdot D_t \quad \forall t = 1, 2, ..., 15$$

**2. Generation limits:** Power output must be within unit limits.

   $$ P_{i}^{min} \cdot U_{i,t} \leq P_{i,t} \leq P_{i}^{max} \cdot U_{i,t} \quad \forall i = 1, 2, ..., 6; \quad \forall t = 1, 2, ..., 15 $$

**3. Ramp-up constraints:** Limit the increase in power output from one period to the next.

   $$ P_{i,t} - P_{i,t-1} \leq RU_i \quad \forall i = 1, 2, ..., 6; \quad \forall t = 2, 3, ..., 15 $$

**4. Ramp-down constraints:** Limit the decrease in power output from one period to the next.

   $$ P_{i,t-1} - P_{i,t}  \leq RD_i \quad \forall i = 1, 2, ..., 6; \quad \forall t = 2, 3, ..., 15 $$

**5. Startup/Shutdown status:**  Constraints to link the binary startup/shutdown variables with the unit on/off status.

   $$ SU_{i,t} \geq U_{i,t} - U_{i,t-1} \quad \forall i = 1, 2, ..., 6; \quad \forall t = 2, 3, ..., 15 $$

   $$ SD_{i,t}  \geq U_{i,t-1} - U_{i,t} \quad \forall i = 1, 2, ..., 6; \quad \forall t = 2, 3, ..., 15 $$

**6. Binary and non-negativity constraints:**

   $$ U_{i,t}, SU_{i,t}, SD_{i,t} \in \{0,1\} \quad \forall i = 1, 2, ..., 6; \quad \forall t = 1, 2, ..., 15 $$
   $$ P_{i,t} \geq 0 \quad \forall i = 1, 2, ..., 6; \quad \forall t = 1, 2, ..., 15 $$ 


## 3. Generate the pyomo code

In [41]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [42]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
I = range(1, 7)  # Index for thermal units
T = range(1, 16) # Index for time periods

# Sample data for parameters - Replace with actual data if available
SU = {1: 100, 2: 150, 3: 80, 4: 120, 5: 90, 6: 110}  # Startup cost (€)
SD = {1: 50, 2: 80, 3: 40, 4: 60, 5: 45, 6: 55}  # Shutdown cost (€)
FC = {1: 200, 2: 250, 3: 180, 4: 220, 5: 190, 6: 210}  # Fixed cost (€)
VC = {(i, t): i * 10 + t * 0.5 for i in I for t in T}  # Variable cost (€/MWh) - Sample data
P_min = {1: 10, 2: 20, 3: 15, 4: 25, 5: 18, 6: 22}  # Minimum power output (MW)
P_max = {1: 100, 2: 150, 3: 80, 4: 120, 5: 90, 6: 110}  # Maximum power output (MW)
RU = {1: 40, 2: 50, 3: 30, 4: 45, 5: 35, 6: 40}  # Maximum ramp-up (MW)
RD = {1: 30, 2: 40, 3: 25, 4: 35, 5: 30, 6: 35}  # Maximum ramp-down (MW)
D = {t: 300 + t * 10 for t in T}  # Total power demand (MW) - Sample data

# Define variables
model.P = pyo.Var(I, T, within=pyo.NonNegativeReals)  # Power output (MW)
model.U = pyo.Var(I, T, within=pyo.Binary)  # Unit commitment status (1 if on, 0 if off)
model.SU = pyo.Var(I, T, within=pyo.Binary)  # Startup status (1 if started up, 0 otherwise)
model.SD = pyo.Var(I, T, within=pyo.Binary)  # Shutdown status (1 if shut down, 0 otherwise)

# Define objective function
model.total_cost = pyo.Objective(
    expr=sum(
        FC[i] * model.U[i, t]
        + VC[i, t] * model.P[i, t]
        + SU[i] * model.SU[i, t]
        + SD[i] * model.SD[i, t]
        for i in I
        for t in T
    ),
    sense=pyo.minimize,
)

# Define constraints

# Power balance constraint
model.power_balance = pyo.ConstraintList()
for t in T:
    model.power_balance.add(
        sum(model.P[i, t] for i in I) >= 1.1 * D[t]
    )

# Generation limits constraint
model.gen_limits = pyo.ConstraintList()
for i in I:
    for t in T:
        model.gen_limits.add(P_min[i] * model.U[i, t] <= model.P[i, t])
        model.gen_limits.add(model.P[i, t] <= P_max[i] * model.U[i, t])

# Ramp-up constraints
model.ramp_up = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.ramp_up.add(model.P[i, t] - model.P[i, t-1] <= RU[i])

# Ramp-down constraints
model.ramp_down = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.ramp_down.add(model.P[i, t-1] - model.P[i, t] <= RD[i])

# Startup/Shutdown status constraints
model.startup_status = pyo.ConstraintList()
model.shutdown_status = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.startup_status.add(model.SU[i, t] >= model.U[i, t] - model.U[i, t-1])
        model.shutdown_status.add(model.SD[i, t] >= model.U[i, t-1] - model.U[i, t])

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
solver.solve(model)

# Print the results
print("Total Cost:", pyo.value(model.total_cost))
print("Power Output:")
for t in T:
    for i in I:
        print(f"Unit {i}, Time {t}: {pyo.value(model.P[i, t])}")

# ... (print other results as needed)
```


## 4. Input problem data and try running the generated code

In [44]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
I = range(1, 7)  # Index for thermal units
T = range(1, 16) # Index for time periods

# Sample data for parameters - Replace with actual data if available
SU = {1: 100, 2: 150, 3: 80, 4: 120, 5: 90, 6: 110}  # Startup cost (€)
SD = {1: 50, 2: 80, 3: 40, 4: 60, 5: 45, 6: 55}  # Shutdown cost (€)
FC = {1: 200, 2: 250, 3: 180, 4: 220, 5: 190, 6: 210}  # Fixed cost (€)
VC = {(i, t): i * 10 + t * 0.5 for i in I for t in T}  # Variable cost (€/MWh) - Sample data
P_min = {1: 10, 2: 20, 3: 15, 4: 25, 5: 18, 6: 22}  # Minimum power output (MW)
P_max = {1: 100, 2: 150, 3: 80, 4: 120, 5: 90, 6: 110}  # Maximum power output (MW)
RU = {1: 40, 2: 50, 3: 30, 4: 45, 5: 35, 6: 40}  # Maximum ramp-up (MW)
RD = {1: 30, 2: 40, 3: 25, 4: 35, 5: 30, 6: 35}  # Maximum ramp-down (MW)
D = {t: 300 + t * 10 for t in T}  # Total power demand (MW) - Sample data

# Define variables
model.P = pyo.Var(I, T, within=pyo.NonNegativeReals)  # Power output (MW)
model.U = pyo.Var(I, T, within=pyo.Binary)  # Unit commitment status (1 if on, 0 if off)
model.SU = pyo.Var(I, T, within=pyo.Binary)  # Startup status (1 if started up, 0 otherwise)
model.SD = pyo.Var(I, T, within=pyo.Binary)  # Shutdown status (1 if shut down, 0 otherwise)

# Define objective function
model.total_cost = pyo.Objective(
    expr=sum(
        FC[i] * model.U[i, t]
        + VC[i, t] * model.P[i, t]
        + SU[i] * model.SU[i, t]
        + SD[i] * model.SD[i, t]
        for i in I
        for t in T
    ),
    sense=pyo.minimize,
)

# Define constraints

# Power balance constraint
model.power_balance = pyo.ConstraintList()
for t in T:
    model.power_balance.add(
        sum(model.P[i, t] for i in I) >= 1.1 * D[t]
    )

# Generation limits constraint
model.gen_limits = pyo.ConstraintList()
for i in I:
    for t in T:
        model.gen_limits.add(P_min[i] * model.U[i, t] <= model.P[i, t])
        model.gen_limits.add(model.P[i, t] <= P_max[i] * model.U[i, t])

# Ramp-up constraints
model.ramp_up = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.ramp_up.add(model.P[i, t] - model.P[i, t-1] <= RU[i])

# Ramp-down constraints
model.ramp_down = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.ramp_down.add(model.P[i, t-1] - model.P[i, t] <= RD[i])

# Startup/Shutdown status constraints
model.startup_status = pyo.ConstraintList()
model.shutdown_status = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.startup_status.add(model.SU[i, t] >= model.U[i, t] - model.U[i, t-1])
        model.shutdown_status.add(model.SD[i, t] >= model.U[i, t-1] - model.U[i, t])

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print the results
print("Total Cost:", pyo.value(model.total_cost))
print("Power Output:")
for t in T:
    for i in I:
        print(f"Unit {i}, Time {t}: {pyo.value(model.P[i, t])}")

Total Cost: 190760.0
Power Output:
Unit 1, Time 1: 100.0
Unit 2, Time 1: 150.0
Unit 3, Time 1: 66.0
Unit 4, Time 1: 25.0
Unit 5, Time 1: 0.0
Unit 6, Time 1: 0.0
Unit 1, Time 2: 100.0
Unit 2, Time 2: 150.0
Unit 3, Time 2: 77.0
Unit 4, Time 2: 25.0
Unit 5, Time 2: 0.0
Unit 6, Time 2: 0.0
Unit 1, Time 3: 100.0
Unit 2, Time 3: 150.0
Unit 3, Time 3: 80.0
Unit 4, Time 3: 33.0000000000001
Unit 5, Time 3: 0.0
Unit 6, Time 3: 0.0
Unit 1, Time 4: 100.0
Unit 2, Time 4: 150.0
Unit 3, Time 4: 80.0
Unit 4, Time 4: 44.0000000000001
Unit 5, Time 4: 0.0
Unit 6, Time 4: 0.0
Unit 1, Time 5: 100.0
Unit 2, Time 5: 150.0
Unit 3, Time 5: 80.0
Unit 4, Time 5: 55.0000000000001
Unit 5, Time 5: 0.0
Unit 6, Time 5: 0.0
Unit 1, Time 6: 100.0
Unit 2, Time 6: 150.0
Unit 3, Time 6: 80.0
Unit 4, Time 6: 66.0000000000001
Unit 5, Time 6: 0.0
Unit 6, Time 6: 0.0
Unit 1, Time 7: 100.0
Unit 2, Time 7: 150.0
Unit 3, Time 7: 80.0
Unit 4, Time 7: 77.0000000000001
Unit 5, Time 7: 0.0
Unit 6, Time 7: 0.0
Unit 1, Time 8: 100.0
U

## 5. Correct the code to verify model viability (optional)

In [46]:
from pyomo.environ import *


import pyomo.environ as pyo

# Sample data - replace with actual data
NU = 6  # Number of thermal units
NT = 15  # Number of time periods

SU =  {'Plant1': 10324, 'Plant2': 5678, 'Plant3': 7802, 'Plant4': 12899, 'Plant5': 4596, 'Plant6': 9076}
SD = {'Plant1': 2673, 'Plant2': 5893, 'Plant3': 982, 'Plant4': 6783, 'Plant5': 2596, 'Plant6': 3561}
FC = {'Plant1': 2000, 'Plant2': 3000, 'Plant3': 2500, 'Plant4': 4000, 'Plant5': 3500, 'Plant6': 4500}  # Sample data
VC = {
    ('Plant1', 1): 20, ('Plant1', 2): 22, ('Plant1', 3): 23, ('Plant1', 4): 24, ('Plant1', 5): 25, 
    ('Plant1', 6): 26, ('Plant1', 7): 27, ('Plant1', 8): 28, ('Plant1', 9): 29, ('Plant1', 10): 30, 
    ('Plant1', 11): 31, ('Plant1', 12): 32, ('Plant1', 13): 33, ('Plant1', 14): 34, ('Plant1', 15): 35,

    ('Plant2', 1): 15, ('Plant2', 2): 16, ('Plant2', 3): 17, ('Plant2', 4): 18, ('Plant2', 5): 19, 
    ('Plant2', 6): 20, ('Plant2', 7): 21, ('Plant2', 8): 22, ('Plant2', 9): 23, ('Plant2', 10): 24, 
    ('Plant2', 11): 25, ('Plant2', 12): 26, ('Plant2', 13): 27, ('Plant2', 14): 28, ('Plant2', 15): 29,

    ('Plant3', 1): 18, ('Plant3', 2): 19, ('Plant3', 3): 20, ('Plant3', 4): 21, ('Plant3', 5): 22, 
    ('Plant3', 6): 23, ('Plant3', 7): 24, ('Plant3', 8): 25, ('Plant3', 9): 26, ('Plant3', 10): 27, 
    ('Plant3', 11): 28, ('Plant3', 12): 29, ('Plant3', 13): 30, ('Plant3', 14): 31, ('Plant3', 15): 32,

    ('Plant4', 1): 25, ('Plant4', 2): 26, ('Plant4', 3): 27, ('Plant4', 4): 28, ('Plant4', 5): 29, 
    ('Plant4', 6): 30, ('Plant4', 7): 31, ('Plant4', 8): 32, ('Plant4', 9): 33, ('Plant4', 10): 34, 
    ('Plant4', 11): 35, ('Plant4', 12): 36, ('Plant4', 13): 37, ('Plant4', 14): 38, ('Plant4', 15): 39,

    ('Plant5', 1): 22, ('Plant5', 2): 23, ('Plant5', 3): 24, ('Plant5', 4): 25, ('Plant5', 5): 26, 
    ('Plant5', 6): 27, ('Plant5', 7): 28, ('Plant5', 8): 29, ('Plant5', 9): 30, ('Plant5', 10): 31, 
    ('Plant5', 11): 32, ('Plant5', 12): 33, ('Plant5', 13): 34, ('Plant5', 14): 35, ('Plant5', 15): 36,

    ('Plant6', 1): 30, ('Plant6', 2): 31, ('Plant6', 3): 32, ('Plant6', 4): 33, ('Plant6', 5): 34, 
    ('Plant6', 6): 35, ('Plant6', 7): 36, ('Plant6', 8): 37, ('Plant6', 9): 38, ('Plant6', 10): 39, 
    ('Plant6', 11): 40, ('Plant6', 12): 41, ('Plant6', 13): 42, ('Plant6', 14): 43, ('Plant6', 15): 44
}

# Technical parameters
Pmin = {'Plant1': 50, 'Plant2': 40, 'Plant3': 30, 'Plant4': 60, 'Plant5': 55, 'Plant6': 65}  # Sample data
Pmax = {'Plant1': 500, 'Plant2': 600, 'Plant3': 550, 'Plant4': 700, 'Plant5': 650, 'Plant6': 750}  # Sample data
RU = {'Plant1': 100, 'Plant2': 120, 'Plant3': 110, 'Plant4': 130, 'Plant5': 125, 'Plant6': 140}  # Sample data
RD = {'Plant1': 90, 'Plant2': 110, 'Plant3': 100, 'Plant4': 120, 'Plant5': 115, 'Plant6': 130}  # Sample data
D =  {1: 280, 2: 327, 3: 400, 4: 388, 5: 501, 6: 600, 7: 800, 8: 927, 9: 705, 10: 502,
     11: 781, 12: 906, 13: 930, 14: 877, 15: 966}  # Sample data


import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
I = ['Plant1', 'Plant2', 'Plant3', 'Plant4', 'Plant5', 'Plant6']  # Index for thermal units
T = range(1, 16) # Index for time periods

# Sample data for parameters - Replace with actual data if available
SU =  {'Plant1': 10324, 'Plant2': 5678, 'Plant3': 7802, 'Plant4': 12899, 'Plant5': 4596, 'Plant6': 9076}
SD = {'Plant1': 2673, 'Plant2': 5893, 'Plant3': 982, 'Plant4': 6783, 'Plant5': 2596, 'Plant6': 3561}
FC = {'Plant1': 2000, 'Plant2': 3000, 'Plant3': 2500, 'Plant4': 4000, 'Plant5': 3500, 'Plant6': 4500}  # Sample data
VC = {
    ('Plant1', 1): 20, ('Plant1', 2): 22, ('Plant1', 3): 23, ('Plant1', 4): 24, ('Plant1', 5): 25, 
    ('Plant1', 6): 26, ('Plant1', 7): 27, ('Plant1', 8): 28, ('Plant1', 9): 29, ('Plant1', 10): 30, 
    ('Plant1', 11): 31, ('Plant1', 12): 32, ('Plant1', 13): 33, ('Plant1', 14): 34, ('Plant1', 15): 35,

    ('Plant2', 1): 15, ('Plant2', 2): 16, ('Plant2', 3): 17, ('Plant2', 4): 18, ('Plant2', 5): 19, 
    ('Plant2', 6): 20, ('Plant2', 7): 21, ('Plant2', 8): 22, ('Plant2', 9): 23, ('Plant2', 10): 24, 
    ('Plant2', 11): 25, ('Plant2', 12): 26, ('Plant2', 13): 27, ('Plant2', 14): 28, ('Plant2', 15): 29,

    ('Plant3', 1): 18, ('Plant3', 2): 19, ('Plant3', 3): 20, ('Plant3', 4): 21, ('Plant3', 5): 22, 
    ('Plant3', 6): 23, ('Plant3', 7): 24, ('Plant3', 8): 25, ('Plant3', 9): 26, ('Plant3', 10): 27, 
    ('Plant3', 11): 28, ('Plant3', 12): 29, ('Plant3', 13): 30, ('Plant3', 14): 31, ('Plant3', 15): 32,

    ('Plant4', 1): 25, ('Plant4', 2): 26, ('Plant4', 3): 27, ('Plant4', 4): 28, ('Plant4', 5): 29, 
    ('Plant4', 6): 30, ('Plant4', 7): 31, ('Plant4', 8): 32, ('Plant4', 9): 33, ('Plant4', 10): 34, 
    ('Plant4', 11): 35, ('Plant4', 12): 36, ('Plant4', 13): 37, ('Plant4', 14): 38, ('Plant4', 15): 39,

    ('Plant5', 1): 22, ('Plant5', 2): 23, ('Plant5', 3): 24, ('Plant5', 4): 25, ('Plant5', 5): 26, 
    ('Plant5', 6): 27, ('Plant5', 7): 28, ('Plant5', 8): 29, ('Plant5', 9): 30, ('Plant5', 10): 31, 
    ('Plant5', 11): 32, ('Plant5', 12): 33, ('Plant5', 13): 34, ('Plant5', 14): 35, ('Plant5', 15): 36,

    ('Plant6', 1): 30, ('Plant6', 2): 31, ('Plant6', 3): 32, ('Plant6', 4): 33, ('Plant6', 5): 34, 
    ('Plant6', 6): 35, ('Plant6', 7): 36, ('Plant6', 8): 37, ('Plant6', 9): 38, ('Plant6', 10): 39, 
    ('Plant6', 11): 40, ('Plant6', 12): 41, ('Plant6', 13): 42, ('Plant6', 14): 43, ('Plant6', 15): 44
}
Pmin = {'Plant1': 50, 'Plant2': 40, 'Plant3': 30, 'Plant4': 60, 'Plant5': 55, 'Plant6': 65}  # Sample data
Pmax = {'Plant1': 500, 'Plant2': 600, 'Plant3': 550, 'Plant4': 700, 'Plant5': 650, 'Plant6': 750}  # Sample data
RU = {'Plant1': 100, 'Plant2': 120, 'Plant3': 110, 'Plant4': 130, 'Plant5': 125, 'Plant6': 140}  # Sample data
RD = {'Plant1': 90, 'Plant2': 110, 'Plant3': 100, 'Plant4': 120, 'Plant5': 115, 'Plant6': 130}  # Sample data
D =  {1: 280, 2: 327, 3: 400, 4: 388, 5: 501, 6: 600, 7: 800, 8: 927, 9: 705, 10: 502,
     11: 781, 12: 906, 13: 930, 14: 877, 15: 966}  # Sample data

# Define variables
model.P = pyo.Var(I, T, within=pyo.NonNegativeReals)  # Power output (MW)
model.U = pyo.Var(I, T, within=pyo.Binary)  # Unit commitment status (1 if on, 0 if off)
model.SU = pyo.Var(I, T, within=pyo.Binary)  # Startup status (1 if started up, 0 otherwise)
model.SD = pyo.Var(I, T, within=pyo.Binary)  # Shutdown status (1 if shut down, 0 otherwise)

# Define objective function
model.total_cost = pyo.Objective(
    expr=sum(
        FC[i] * model.U[i, t]
        + VC[i, t] * model.P[i, t]
        + SU[i] * model.SU[i, t]
        + SD[i] * model.SD[i, t]
        for i in I
        for t in T
    ),
    sense=pyo.minimize,
)

# Define constraints

# Power balance constraint
model.power_balance = pyo.ConstraintList()
for t in T:
    model.power_balance.add(
        sum(model.P[i, t] for i in I) >= 1.1 * D[t]
    )

# Generation limits constraint
model.gen_limits = pyo.ConstraintList()
for i in I:
    for t in T:
        model.gen_limits.add(P_min[i] * model.U[i, t] <= model.P[i, t])
        model.gen_limits.add(model.P[i, t] <= P_max[i] * model.U[i, t])

# Ramp-up constraints
model.ramp_up = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.ramp_up.add(model.P[i, t] - model.P[i, t-1] <= RU[i])

# Ramp-down constraints
model.ramp_down = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.ramp_down.add(model.P[i, t-1] - model.P[i, t] <= RD[i])

# Startup/Shutdown status constraints
model.startup_status = pyo.ConstraintList()
model.shutdown_status = pyo.ConstraintList()
for i in I:
    for t in range(2, 16):
        model.startup_status.add(model.SU[i, t] >= model.U[i, t] - model.U[i, t-1])
        model.shutdown_status.add(model.SD[i, t] >= model.U[i, t-1] - model.U[i, t])

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print the results
print("Total Cost:", pyo.value(model.total_cost))
print("Power Output:")
for t in T:
    for i in I:
        print(f"Unit {i}, Time {t}: {pyo.value(model.P[i, t])}")

KeyError: 'Plant1'

## 6. Print the responses

In [47]:
print(response.text)

## Indices:

*  $i$: Index for thermal units, $i = 1, 2, ..., 6$
* $t$: Index for time periods, $t = 1, 2, ..., 15$ 

## Parameters:

* $SU_i$: Startup cost of unit $i$ (€) 
* $SD_i$: Shutdown cost of unit $i$ (€)
* $FC_i$: Fixed cost of unit $i$ (€) 
* $VC_{i,t}$: Variable cost of unit $i$ at time $t$ (€/MWh)
* $P_{i}^{min}$: Minimum power output of unit $i$ (MW)
* $P_{i}^{max}$: Maximum power output of unit $i$ (MW)
* $RU_i$: Maximum power increment of unit $i$ from one period to the next (MW) 
* $RD_i$: Maximum power decrement of unit $i$ from one period to the next (MW)
* $D_t$: Total power demand at time $t$ (MW)

## Variables:

* $P_{i,t}$: Power output of unit $i$ at time $t$ (MW)
* $U_{i,t}$: Binary variable indicating if unit $i$ is running at time $t$ (1 if running, 0 otherwise)
* $SU_{i,t}$: Binary variable indicating if unit $i$ is started up at time $t$ (1 if started, 0 otherwise)
* $SD_{i,t}$: Binary variable indicating if unit $i$ is shut down at time $t$ (1 if shut down

In [48]:
print(response2.text)

$$
\text{Minimize} \sum_{i=1}^{6} \sum_{t=1}^{15} (FC_i \cdot U_{i,t} + VC_{i,t} \cdot P_{i,t} + SU_i \cdot SU_{i,t} + SD_i \cdot SD_{i,t}) 
$$



In [49]:
print(response3.text)

## Constraints:

**1. Power balance:** The total power generated must meet demand plus a 10% reserve margin.
   
   $$ \sum_{i=1}^{6} P_{i,t} \geq 1.1 \cdot D_t \quad \forall t = 1, 2, ..., 15$$

**2. Generation limits:** Power output must be within unit limits.

   $$ P_{i}^{min} \cdot U_{i,t} \leq P_{i,t} \leq P_{i}^{max} \cdot U_{i,t} \quad \forall i = 1, 2, ..., 6; \quad \forall t = 1, 2, ..., 15 $$

**3. Ramp-up constraints:** Limit the increase in power output from one period to the next.

   $$ P_{i,t} - P_{i,t-1} \leq RU_i \quad \forall i = 1, 2, ..., 6; \quad \forall t = 2, 3, ..., 15 $$

**4. Ramp-down constraints:** Limit the decrease in power output from one period to the next.

   $$ P_{i,t-1} - P_{i,t}  \leq RD_i \quad \forall i = 1, 2, ..., 6; \quad \forall t = 2, 3, ..., 15 $$

**5. Startup/Shutdown status:**  Constraints to link the binary startup/shutdown variables with the unit on/off status.

   $$ SU_{i,t} \geq U_{i,t} - U_{i,t-1} \quad \forall i = 1, 2, ..., 6; \q

In [50]:
print(response4.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
I = range(1, 7)  # Index for thermal units
T = range(1, 16) # Index for time periods

# Sample data for parameters - Replace with actual data if available
SU = {1: 100, 2: 150, 3: 80, 4: 120, 5: 90, 6: 110}  # Startup cost (€)
SD = {1: 50, 2: 80, 3: 40, 4: 60, 5: 45, 6: 55}  # Shutdown cost (€)
FC = {1: 200, 2: 250, 3: 180, 4: 220, 5: 190, 6: 210}  # Fixed cost (€)
VC = {(i, t): i * 10 + t * 0.5 for i in I for t in T}  # Variable cost (€/MWh) - Sample data
P_min = {1: 10, 2: 20, 3: 15, 4: 25, 5: 18, 6: 22}  # Minimum power output (MW)
P_max = {1: 100, 2: 150, 3: 80, 4: 120, 5: 90, 6: 110}  # Maximum power output (MW)
RU = {1: 40, 2: 50, 3: 30, 4: 45, 5: 35, 6: 40}  # Maximum ramp-up (MW)
RD = {1: 30, 2: 40, 3: 25, 4: 35, 5: 30, 6: 35}  # Maximum ramp-down (MW)
D = {t: 300 + t * 10 for t in T}  # Total power demand (MW) - Sample data

# Define variables
model.P = pyo.Var(I, T, within=pyo